In [1]:
from pyspark_llap.sql.session import HiveWarehouseSession
from pyspark.sql import SparkSession
#spark = SparkSession.builder.appName("LLAP_Test").enableHiveSupport().config("spark.sql.hive.llap", "true").getOrCreate()
spark = SparkSession.builder.appName("Hive_ACID_warehouseconnector").enableHiveSupport().getOrCreate()
hive = HiveWarehouseSession.session(spark).build()

In [2]:
from pyspark.sql.functions import udf, col, lit, array
import pyspark.sql.functions as F
from pyspark.sql import Row
from pyspark.sql.types import StructType, StructField, IntegerType, StringType, DateType, FloatType

# from pyspark_llap.sql.session import HiveWarehouseSession
# from pyspark.sql import SparkSession
# hive = HiveWarehouseSession.session(spark).build()


import time
import pandas as pd
import numpy as np
import datetime
import math
import scipy
import matplotlib.pyplot as plt
import statsmodels.api as sm
import statsmodels.formula.api as smf
import statsmodels
import subprocess
import sys
import pyspark.sql as SQL
from pyspark.sql.functions import concat_ws
from pyspark.sql.functions import avg
from pyspark import SparkConf
from pyspark.sql import HiveContext
from pyspark import SparkContext
from pyspark.sql.functions import col,row_number
from pyspark.sql.types import *
from pyspark import SparkContext, SparkConf
from pyspark.sql import SQLContext
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, udf
import pyspark.sql.functions as F
from pyspark.sql.window import Window
from pyspark.sql.types import Row
from pyspark.sql.functions import to_timestamp,unix_timestamp,from_unixtime
from pyspark.sql.types import DoubleType, FloatType, BooleanType, LongType, IntegerType, TimestampType, StringType, StructType, StructField, ArrayType

sqlContext.setConf("spark.sql.shuffle.partitions", "2500")

In [ ]:
numerical_cols = ['BrkTot_Tq_Actl', 'Veh_V_ActlEng', 'ApedPos_Pc_ActlArb', 'EngAout_N_Actl', 'HvacEvap_Te_Actl', 'HvacEvap_Te_Rq', 'BattULo_U_Actl', 'AirCondFluidHi_P_Actl', 'AirAmb_Te_ActlFilt', 'Outside_Air_Temp_Stat', 'OdometerMasterValue', 'VehLong2_A_Actl']
cat_cols = ['HvacBlwrFront_D_Stat','GearLvrPos_D_Actl', 'HvacAirCond_B_Rq', 'CoolantFanStepAct']

In [3]:
%%time
df = spark.sql("select * from ccbdd.2019f150ss where cvdcus_timestamp_s_3 BETWEEN '2019-05-01' and '2019-08-30'")\
.withColumn('epoch', F.round(unix_timestamp(col('cvdcus_timestamp_s_3')), 0))\
.withColumn('datetime',to_timestamp(from_unixtime(col('epoch')), 'yyyy-MM-dd HH:mm:ss'))\
.groupBy('cvdcus_vin_d_3','epoch','datetime')\
.pivot('cvdcus_dcd_sig_n_x_3')\
.agg(avg('cvdcus_dcd_sig_val_str_x_3'))\
.orderBy(['epoch'], ascending=[1])

KeyboardInterrupt: 

In [4]:
%%time
pivotted = df.toPandas()

NameError: name 'df' is not defined

In [16]:
df = pd.read_csv('2019f150VIN1FTEW1C53KFB01079_subset.csv')
spark_df = spark.createDataFrame(df)

In [37]:
# pivotted = rdd.withColumn('epoch', F.round(unix_timestamp(col('cvdcus_timestamp_s_3')), 0))\
# .withColumn('datetime',to_timestamp(from_unixtime(col('epoch')), 'yyyy-MM-dd HH:mm:ss'))\
# .groupBy('cvdcus_vin_d_3','epoch','datetime')\
# .pivot('cvdcus_dcd_sig_n_x_3')\
# .agg(avg('cvdcus_dcd_sig_val_str_x_3'))\
# .orderBy(['epoch'], ascending=[1]).toPandas()

In [44]:
numericals = spark_df.filter(col('cvdcus_dcd_sig_n_x_3').isin(['BrkTot_Tq_Actl', 'VehLong2_A_Actl', 'EngAout_N_Actl', 'ApedPos_Pc_ActlArb']))\
.filter(col('cvdcus_dcd_sig_val_str_x_3').rlike('[0-9]+')).withColumn('time', to_timestamp(col('cvdcus_timestamp_s_3'), 'yyyy-MM-dd HH:mm:ss')).withColumn('epoch', F.round(unix_timestamp(col('time')), 0))\
.withColumn('datetime',to_timestamp(from_unixtime(col('epoch')), 'yyyy-MM-dd HH:mm:ss'))\
.groupBy('epoch','datetime')\
.pivot('cvdcus_dcd_sig_n_x_3')\
.agg(avg('cvdcus_dcd_sig_val_str_x_3'))\
.orderBy(['epoch'], ascending=[1]).toPandas()

In [36]:
t = spark_df.filter(col('cvdcus_dcd_sig_n_x_3').isin(['BrkTot_Tq_Actl', 'VehLong2_A_Actl', 'EngAout_N_Actl', 'ApedPos_Pc_ActlArb']))\
.filter(col('cvdcus_dcd_sig_val_str_x_3').rlike('[0-9]+')).toPandas()

In [45]:
numericals

,epoch,datetime,ApedPos_Pc_ActlArb,BrkTot_Tq_Actl,EngAout_N_Actl,VehLong2_A_Actl
0,None,None,5.7825,311.737063,989.202318,-0.038172
